In [1]:
from backend.functions import *
from backend.classes import *

C:\Users\darcy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
with open('data/Frankfurt.json', encoding="utf-8") as f:
    frankfurt = json.load(f)

with open('data/Monterrey.json', encoding="utf-8") as f:
    monterrey = json.load(f)

with open('data/Pittsburgh.json', encoding="utf-8") as f:
    pittsburgh = json.load(f)

with open('data/Milwaukee.json', encoding="utf-8") as f:
    milwaukee = json.load(f)

with open('data/BeloHorizonte.json', encoding="utf-8") as f:
    belohorizonte = json.load(f)

In [4]:
for item in frankfurt:
    item['tournament']='Frankfurt'
    item['date'] = '2025-09-14'

for item in monterrey:
    item['tournament']='Monterrey'
    item['date'] = '2025-09-14'

for item in pittsburgh:
    item['tournament']='Pittsburgh'
    item['date'] = '2025-09-21'

for item in milwaukee:
    item['tournament']='Milwaukee'
    item['date'] = '2025-10-12'

for item in belohorizonte:
    item['tournament']='Belo Horizonte'
    item['date'] = '2025-10-12'

In [5]:
data = frankfurt + monterrey + pittsburgh + milwaukee + belohorizonte

In [6]:
processed = process_tournament_data(data)

In [7]:
processed

,Competition,Name,Nationality,Wins,Losses,Date,Corviknight,Sableye,Klefki,Comfey,...,Quagsire,Reuniclus,Sunflora,Pawmot,Florges,Palafin,Kommo-o,Gloom,Meganium,Bronzong
0,Frankfurt,Emanuele Briganti,IT,14,2,2025-09-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Frankfurt,Víctor Medina,ES,13,3,2025-09-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Frankfurt,Shohei Kimura,JP,12,3,2025-09-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Frankfurt,Davide Carrer,IT,12,3,2025-09-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Frankfurt,Fabian Braun,DE,12,2,2025-09-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,Belo Horizonte,Guilherme Fernandes,BR,0,2,2025-10-12,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2660,Belo Horizonte,JESSICA DA SILVA,BR,0,2,2025-10-12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2661,Belo Horizonte,Sávio Lima Soares,BR,0,2,2025-10-12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2662,Belo Horizonte,Gael Garcia,BR,0,3,2025-10-12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
recent = process_tournament_data(milwaukee+belohorizonte)

In [27]:
clusterer = PokemonTeamClustering(recent)


clusterer.select_features(method='frequency', threshold=0.03)

clusterer.select_features(method='variance', threshold=0.04)

# Weight Pokemon more heavily than moves
clusterer.normalize_features(pokemon_weight=1.1, move_weight=0.6)

optimal_clusters = clusterer.find_optimal_clusters(max_clusters=20)

best_n = optimal_clusters.loc[optimal_clusters['silhouette_score'].idxmax(), 'n_clusters']

clusterer.cluster_teams(n_clusters=best_n)


print(clusterer.identify_archetypes())

{'Archetype 1: Basculegion [Male], Pelipper, Archaludon, Incineroar': {'core_pokemon': ['Basculegion [Male]', 'Pelipper', 'Archaludon', 'Incineroar'], 'key_moves': [], 'team_count': 61, 'frequency': 0.06823266219239374}, 'Archetype 2: Sneasler, Rillaboom, Ninetales [Alolan Form], Ursaluna [Bloodmoon], Gholdengo': {'core_pokemon': ['Sneasler', 'Rillaboom', 'Ninetales [Alolan Form]', 'Ursaluna [Bloodmoon]', 'Gholdengo'], 'key_moves': [], 'team_count': 40, 'frequency': 0.0447427293064877}, 'Archetype 3: Indeedee [Male], Tyranitar, Sneasler': {'core_pokemon': ['Indeedee [Male]', 'Tyranitar', 'Sneasler'], 'key_moves': [], 'team_count': 30, 'frequency': 0.03355704697986577}, 'Archetype 4: Ursaluna, Incineroar, Amoonguss, Gholdengo, Porygon2': {'core_pokemon': ['Ursaluna', 'Incineroar', 'Amoonguss', 'Gholdengo', 'Porygon2'], 'key_moves': [], 'team_count': 28, 'frequency': 0.03131991051454139}, 'Archetype 5: Ursaluna, Indeedee [Female], Torkoal': {'core_pokemon': ['Ursaluna', 'Indeedee [Female

In [28]:
clusterer.plot_cluster_scatter('tsne')

In [29]:
frequencies = clusterer.identify_archetypes()

In [33]:
freq = 0.0
for archetype, teams in frequencies.items():
    freq += teams['frequency']

In [34]:
freq

0.26174496644295303